#ID0309 - Aprendizaje Estadístico

### Otoño 2023

### ID0309_Lab_1-6

**Enrique Naredo García**

<font size = 2>
©️ Todos los derechos reservados. All rights reserved.

*Nota: El presente documento es una herramienta diseñada única y exclusivamente para los estudiantes de la asignatura arriba mencionada. Se recuerda no compartir esta información fuera de los integrantes registrados en este curso. La reproducción total o parcial de este documento requiere autorización por escrito del titular del copyright.*
</font>

#Reducción de dimensionalidad

##Clasificación Binaria

In [13]:
# librerías
from sklearn.datasets import make_classification

Generamos un conjunto de datos con una etiqueta binaria.
* Es decir, una etiqueta con sólo dos valores posibles: 0 o 1.
* Para hacer esto, establezca el valor del parámetro n_classes en 2.
* Crearemos un conjunto de datos con 1000 observaciones.
* Contará con cinco artículos, de los cuales tres serán informativos.
* Las otras dos funciones serán redundantes.

In [14]:
# Genera un conjunto de datos
X, y = make_classification(
    # 1000 observaciones
    n_samples=1000,
    # 5 características (dimensiones)
    n_features=5,
    # 3 características 'utiles'
    n_informative=3,
    # número de etiquetas para las dos clases
    n_classes=2,
    # para obtener los mismos resultados
    random_state=77
)


Si no especificas ```random_state``` en tu código cada vez que lo ejecutas.
* Se genera un nuevo valor aleatorio.
* Los conjuntos de datos de entrenamiento y prueba tendrán valores diferentes cada vez.
* Para todos los conjuntos de datos aleatorios, cada uno asigna un valor de estado_aleatorio.
* Significa que un valor de estado_aleatorio tiene un conjunto de datos fijo.
* Esto es que cada vez que ejecutamos código con valor random_state, digamos 1, producirá los mismos conjuntos de datos divididos.

Convirtamos la salida de make_classification() en un DataFrame de pandas.
* Es más fácil analizar un DataFrame que las matrices NumPy sin procesar.

In [15]:
import pandas as pd

# Crea un DataFrame
dataset = pd.DataFrame(X)
# nombra las características
dataset.columns = ['X1', 'X2', 'X3', 'X4', 'X5']
# y agrega una etiqueta como columna
dataset['y'] = y

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      1000 non-null   float64
 1   X2      1000 non-null   float64
 2   X3      1000 non-null   float64
 3   X4      1000 non-null   float64
 4   X5      1000 non-null   float64
 5   y       1000 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 47.0 KB


Como se esperaba, el conjunto de datos tiene 1000 observaciones, cinco características (X1, X2, X3, X4 y X5) y la etiqueta de destino correspondiente (y).

* Habíamos establecido el parámetro n_informative en 3.
* Por lo tanto, solo las tres primeras características (X1, X2, X3) son importantes.
* Los otros, X4 y X5, son redundantes.1

In [16]:
# verifica los valores únicos para la etiqueta "y"
dataset['y'].value_counts()

1    503
0    497
Name: y, dtype: int64

La etiqueta tiene sólo dos valores posibles (0 y 1).
* Entonces es un conjunto de datos de clasificación binaria.
* Además, los recuentos de ambos valores son aproximadamente iguales.
* Así, la etiqueta tiene clases equilibradas.

In [17]:
# aquí están las primeras cinco observaciones
dataset.head()

,X1,X2,X3,X4,X5,y
0,-1.436084,-1.369394,0.858954,-0.325778,0.976911,1
1,1.229934,-1.134827,-1.747461,0.790104,-1.242959,0
2,-1.556988,-0.921732,0.944404,-0.365890,0.964439,1
3,-0.667602,-1.100769,1.103220,-0.483515,1.125059,1
4,1.025707,-0.316636,-0.897722,0.383591,-0.690592,0


##Ejemplo de clasificador

Utilizaremos un modelo "RandomForestClassifier" con hiperparámetros predeterminados.
* Usaremos [validación cruzada](https://es.wikipedia.org/wiki/Validaci%C3%B3n_cruzada) y mediremos la puntuación del modelo en métricas de clasificación clave.

In [18]:
# librerías
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate


In [19]:
# inicializa el clasificador (modelo)
classifier = RandomForestClassifier()

In [20]:
# ejecuta validación cruzada con 10 grupos
scores = cross_validate(
    classifier, X, y, cv=10,
    # lista de metricas
    scoring=['accuracy', 'precision', 'recall', 'f1']
)

In [21]:
# verifica los valores de cada metrica
scores = pd.DataFrame(scores)
scores.mean().round(4)

fit_time          0.1670
score_time        0.0080
test_accuracy     0.9660
test_precision    0.9600
test_recall       0.9740
test_f1           0.9666
dtype: float64

## Dataset más difícil

Creamos un conjunto de datos que no será tan fácil de clasificar.

Puede controlar el nivel de dificultad de un conjunto de datos utilizando los siguientes parámetros de la función make_classification: flip_y: agrega ruido al voltear algunas etiquetas.

* Por ejemplo, cambie algunas etiquetas de 0 a 1 y viceversa.
* Un valor más alto voltea más etiquetas y, por tanto, añade más ruido. * El valor predeterminado es 0,01.
* class_sep: controla el espacio entre las clases de etiquetas.
* Un valor menor reduce el espacio y, por tanto, dificulta la clasificación.
* El valor predeterminado es 1,0.
* Usaremos un valor más alto para flip_y y un valor más bajo para class_sep para crear un conjunto de datos desafiante

In [22]:
# crea un muevo conjunto de datos
X2, y2 = make_classification(
    # similar al anterior
    n_samples=1000, n_features=5, n_informative=3, n_classes=2,
    # agrega ruido
    flip_y=0.1,
    # a menor valor reduce la separación entre clases
    class_sep=0.5
)


In [23]:
# verifica la distribución de clases
pd.DataFrame(y2).value_counts()

1    506
0    494
dtype: int64

Clasificación del conjunto de datos más complejo.
* Construyamos nuevamente un modelo RandomForestClassifier con hiperparámetros predeterminados.
* Esta vez, entrenaremos el modelo con el conjunto de datos más complejo que acabamos de crear.

In [24]:
# usamos el modelo de Random Forest
classifier2 = RandomForestClassifier()

In [25]:
# metricas
scores2 = cross_validate(
    classifier2, X2, y2, cv=10,
    scoring=['accuracy', 'precision', 'recall', 'f1']
)

In [26]:
# verifica los valores de cada metrica
scores2 = pd.DataFrame(scores2)
scores2.mean()

fit_time          0.192358
score_time        0.007947
test_accuracy     0.785000
test_precision    0.770133
test_recall       0.826118
test_f1           0.795408
dtype: float64

La exactitud, la precisión, la recuperación y la puntuación F1 de este modelo rondan el 75-76%.
* Esa es una fuerte disminución del 88% para el modelo entrenado con el conjunto de datos más sencillo.
* ¡Los valores personalizados para los parámetros flip_y y class_sep funcionaron!
* Crearon un conjunto de datos que es más difícil de clasificar.

## Conjunto de datos desbalanceado

Hasta ahora, hemos creado conjuntos de datos con un número aproximadamente igual de observaciones asignadas a cada clase de etiqueta.

* ¿Qué pasaría si quisieras un conjunto de datos con clases desequilibradas?
* Es decir, ¿un conjunto de datos donde una de las clases de etiquetas ocurre raramente?
* Puede utilizar las ponderaciones de los parámetros para controlar la proporción de observaciones asignadas a cada clase.

In [27]:
# crea un conjunto de datos
X3, y3 = make_classification(
    # parametros usuales
    n_samples=1000, n_features=5, n_informative=3, n_classes=2,
    # 97% clase 0 y 3% el resto
    weights=[0.97],
)

Confirmamos el debalance de clases.


In [28]:
# verifica la distribución de clases
pd.DataFrame(y3).value_counts()

0    965
1     35
dtype: int64

Efectivamente, make_classification() asignó aproximadamente el 3% de las observaciones a la clase 1.

Clasificación de conjuntos de datos desequilibrados.

Como antes, crearemos un modelo RandomForestClassifier con hiperparámetros predeterminados.

Y luego entrenarlo en el conjunto de datos desequilibrado.

In [29]:
# usamos el modelo de Random Forest
classifier3 = RandomForestClassifier()

In [30]:
# aplicamos validación cruzada
scores3 = cross_validate(
    classifier3, X3, y3, cv=10,
    scoring=['accuracy', 'precision', 'recall', 'f1']
);


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
# metricas
scores3 = pd.DataFrame(scores)
scores3.mean()

fit_time          0.167021
score_time        0.008011
test_accuracy     0.966000
test_precision    0.960012
test_recall       0.974039
test_f1           0.966618
dtype: float64

La mayoría de las observaciones tienen No como resultado, por lo que es la clase mayoritaria.
Y sí, es la clase minoritaria, ya que rara vez ocurre.

Cuando la variable de salida tiene una disparidad tan amplia en la frecuencia de clases, decimos que la salida tiene clases desequilibradas y que el conjunto de datos está desequilibrado.

¿Qué pasa si usa este conjunto de datos para entrenar un modelo de clasificación? Es posible que el modelo no tenga suficientes observaciones con el resultado Sí para aprender a identificar transacciones fraudulentas.

La precisión será un mal indicador del desempeño de un modelo para conjuntos de datos tan desequilibrados. Siga leyendo para averiguar por qué.

##Conjunto de datos multiclase

Hasta ahora, hemos creado etiquetas con sólo dos valores posibles.

¿Qué pasaría si quisieras experimentar con conjuntos de datos multiclase donde la etiqueta puede tomar más de dos valores?

* Puedes hacerlo usando el parámetro n_classes.
* El siguiente código creará una etiqueta con 3 clases.

In [32]:
# Multiclase
X4, y4 = make_classification(
    # mismos parametros
    n_samples=1000, n_features=5, n_informative=3,
    # ahora usamos 3 clases
    n_classes=3,
)

Confirmemos que la etiqueta efectivamente tiene 3 clases (0, 1 y 2).
* Los tres tienen aproximadamente el mismo número de observaciones.
* Es decir, tenemos las clases equilibradas.

In [33]:
# Datos con 3 clases (0, 1 y 2):
pd.DataFrame(y4).value_counts()

2    335
1    334
0    331
dtype: int64

##Conjunto de datos desequilibrado multiclase

Puede crear fácilmente conjuntos de datos con etiquetas multiclase desequilibradas.
* Simplemente use el parámetro n_classes junto con los pesos.
* En el siguiente código, le pedimos a make_classification() que asigne solo el 4% de las observaciones a la clase 0.
* Y divida el resto de las observaciones en partes iguales entre las clases restantes (48% cada una).

In [34]:
# creamos otro conjunto de datos
X5, y5 = make_classification(
    # same parameters as usual
    n_samples=1000, n_features=5, n_informative=3,
    # create target label with 3 classes
    n_classes=3,
    # assign 4% of rows to class 0, 48% to class 1
    # and the rest to class 2
    weights=[0.04, 0.48]
)

Confirmemos el desequilibrio de clases.
* La clase 0 tiene solo 44 observaciones de 1000.

In [35]:
# verifica la distribución de clases
pd.DataFrame(y5).value_counts()

1    478
2    476
0     46
dtype: int64